In [1]:
# def import_wrapper(name, *args, imp=__import__):
#     print("importing ", name)
#     return imp(name, *args)

In [2]:
# import builtins
# builtins.__import__ = import_wrapper
# import collections

In [1]:
def read_int(handle):
    return int.from_bytes(handle.read(4), "little")

In [2]:
import useful.foo as foo

useful.foo
useful.__init__.py


In [3]:
foo.__cached__

'/home/leo/ME/computer-science/CSC Python/Modules and packages/useful/__pycache__/foo.cpython-37.pyc'

In [4]:
handle = open(foo.__cached__, "rb")

In [5]:
magic = read_int(handle)
print(magic)

168627522


In [6]:
mtime = read_int(handle)
print(mtime)

0


In [7]:
import time

In [12]:
print(time.asctime(time.localtime(mtime)))

Thu Jan  1 03:00:00 1970


In [9]:
read_int(handle)

1600703472

In [10]:
import marshal

In [11]:
marshal.loads(handle.read())

ValueError: bad marshal data (unknown type code)

In [13]:
import sys

In [15]:
sys.modules["sys"]

<module 'sys' (built-in)>

In [16]:
id(sys.modules["sys"])

140320786374000

In [24]:
builtin_finder, _, path_finder, _ = sys.meta_path

In [25]:
builtin_finder.find_spec("itertools")

ModuleSpec(name='itertools', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in')

In [26]:
builtin_finder.find_spec("enum")

In [27]:
path_finder.find_spec("enum")

ModuleSpec(name='enum', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7f9efa8134d0>, origin='/home/leo/Conda/lib/python3.7/enum.py')

In [28]:
path_finder.find_spec("math")

ModuleSpec(name='math', loader=<_frozen_importlib_external.ExtensionFileLoader object at 0x7f9ede88b490>, origin='/home/leo/Conda/lib/python3.7/lib-dynload/math.cpython-37m-x86_64-linux-gnu.so')

In [29]:
spec = path_finder.find_spec("collections")

In [30]:
spec.name

'collections'

In [31]:
spec.origin

'/home/leo/Conda/lib/python3.7/collections/__init__.py'

In [32]:
spec.cached

'/home/leo/Conda/lib/python3.7/collections/__pycache__/__init__.cpython-37.pyc'

In [33]:
spec.parent

'collections'

In [34]:
spec.submodule_search_locations

['/home/leo/Conda/lib/python3.7/collections']

In [35]:
spec.loader

Automatic installation:

In [36]:
import subprocess
import sys
from importlib.util import find_spec
from importlib.abc import MetaPathFinder

In [37]:
class AutoInstall(MetaPathFinder):
    _loaded = set()
    
    @classmethod
    def find_spec(cls, name, path=None, target=None):
        if path is None and name not in cls._loaded:
            print("Installing", name)
            cls._loaded.add(name)
            try:
                subprocess.check_output([sys.executable, "-m", "pip", "install", name])
                return find_spec(name)
            except Exception:
                print("Failed")
        return None

In [38]:
sys.executable

'/home/leo/Conda/bin/python'

In [40]:
sys.path_hooks

[zipimport.zipimporter,
 <function _frozen_importlib_external.FileFinder.path_hook.<locals>.path_hook_for_FileFinder(path)>]

Remote import:

In [41]:
import re
import sys
from urllib.request import urlopen

def url_hook(url):
    if not url.startswith(("http", "https")):
        raise ImportError
    with urlopen(url) as page:
        data = page.read().decode("utf-8")
    filenames = re.findall("[a-zA-Z_][a-zA-Z0-9_]*.py", data)
    modnames = {name[:-3] for name in filenames}
    return URLFinder(url, modnames)

sys.path_hooks.append(url_hook)

In [43]:
from importlib.abc import PathEntryFinder
from importlib.util import spec_from_loader

In [44]:
class URLFinder(PathEntryFinder):
    def __init__(self, url, available):
        self.url = url
        self.available = available
        
    def find_spec(self, name, target=None):
        if name in self.available:
            origin = "{}/{}.py".format(self.url, name)
            loader = URLLoader()
            return spec_from_loader(name, loader, origin=origin)
        else:
            return None

In [45]:
from urllib.request import urlopen

In [46]:
class URLLoader:
    def create_module(self, target):
        return None
    
    def exec_module(self, module):
        with urlopen(module.__spec__.origin) as page:
            source = page.read()
        code = compile(source, module.__spec__.origin, mode="exec")
        exec(code, module.__dict__)